In [1]:
import sys 
sys.path.append('..')
import numpy as np
import onnxruntime as ort
from tqdm import trange
from utils.video import read_video, transform_img

In [2]:
frames = read_video("../examples/sample_video_ecamera.hevc")

In [3]:
frames = np.array([transform_img(x) for x in frames])

In [4]:
# load model session
options = ort.SessionOptions()
provider = 'CUDAExecutionProvider'
session = ort.InferenceSession('../gpt2m/encoder.onnx', options, [provider])

2023-08-30 15:59:05.017313715 [W:onnxruntime:, session_state.cc:1169 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2023-08-30 15:59:05.017329214 [W:onnxruntime:, session_state.cc:1171 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [5]:
# print shapes
input_shapes = {i.name: i.shape for i in session.get_inputs()}
output_shapes = {i.name: i.shape for i in session.get_outputs()}
print('input shapes : ', input_shapes)
print('output shapes: ', output_shapes)

input shapes :  {'big_img': ['b', 3, 128, 256]}
output shapes:  {'encoding_indices': ['b', 8, 16]}


In [6]:
# encoding loop
tokens = []
for i in trange(len(frames)):
  outputs = session.run(None, {'big_img': frames[i].transpose(2,0,1)[None].astype(np.float32)})
  outputs = {o.name: x for o,x in zip(session.get_outputs(), outputs)}
  tokens.append(outputs['encoding_indices'].ravel())

100%|██████████| 1200/1200 [00:10<00:00, 112.65it/s]


In [7]:
# save the tokens! now head over to the decoding notebook
tokens = np.array(tokens)
np.save("../examples/tokens.npy", tokens)